In [31]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory, TerminationCondition
import psycopg2

In [32]:
conexion = psycopg2.connect(
        dbname="buap",
        user="postgres",
        password="contrasena",
        host="localhost",
        port="5432"
    )
cursor = conexion.cursor()
datos = {}

In [33]:
# Profesores (con nombre)
cursor.execute("SELECT id_profesor, nombre, id_contrato FROM Profesor  ")
profesores = cursor.fetchall()
datos['Id_Profesores'] = [row[0] for row in profesores] # Lista de IDs de profesores
datos['nombres_profesor'] = {row[0]: row[1] for row in profesores} # Diccionario de nombres de profesores
datos['Tipo_Contrato'] = {row[0]: row[2] for row in profesores} # Diccionario de tipo de contrato de cada profesor

In [34]:
#TIpos de contratos 
cursor.execute("SELECT id_contrato, horas_minimas, horas_maximas FROM Contrato WHERE id_contrato = ANY(%s)", (list(datos['Tipo_Contrato'].values()),))
contratos_detalles = cursor.fetchall()
datos['Detalles_Contratos'] = {row[0]: {'min': row[1], 'max': row[2]} for row in contratos_detalles}  # Detalles por id_contrato
datos['Detalles_Contratos'][4] = {'min': 0, 'max': 99}  # Contrato de profesor imaginario

In [35]:
# Materias (con nombre)
cursor.execute("SELECT id_materia, nombre, horas_por_semana, id_tipo_clase, id_programa_educativo FROM Materia limit 10")
materias = cursor.fetchall()
datos['id_Materias'] = [row[0] for row in materias] # Lista de IDs de materias
datos['nombres_materia'] = {row[0]: row[1] for row in materias} # Diccionario de nombres de materias
datos['Horas_Semana_Materia'] = {row[0]: row[2] for row in materias} # Diccionario de horas por semana de cada materia
datos['Tipo_Clase'] = {row[0]: row[3] for row in materias} # Diccionario de tipo de clase de cada materia
datos['Programa_Educativo_Materia'] = {row[0]: row[4] for row in materias} # Diccionario de programa educativo de cada materia

In [36]:
# Aulas
cursor.execute("SELECT id_aula, id_edificio, id_tipo_clase FROM Aula ")
rows = cursor.fetchall()

datos['Aulas'] = [(row[0], row[1]) for row in rows] # Lista de IDs compuestos (id_aula, id_edificio)
datos['Tipo_Aula'] = {(row[0], row[1]): row[2] for row in rows}# Diccionario: clave compuesta (id_aula, id_edificio) -> tipo clase

# Edificios 
cursor.execute("SELECT id_edificio, nombre FROM Edificio")
rows = cursor.fetchall()

datos['Id_Edificios'] = [row[0] for row in rows]          # Lista con todos los id_edificio
datos['Edificios'] = {row[0]: row[1] for row in rows}     # Diccionario id_edificio -> nombre



# Bloques horarios
datos['Dia'] = list(range(1, 6)) # Lunes a Viernes
datos['Hora'] = list(range(1, 11)) # 10 bloques horarios por día/ de 8 am a 6 pm

In [37]:
# Programa Educativo
cursor.execute("SELECT id_programa_educativo, nombre, abreviatura, id_unidad_academica FROM Programa_Educativo")
rows = cursor.fetchall()

# Lista de IDs
datos['Id_Programas_Educativos'] = [row[0] for row in rows]

# Diccionario completo: id_programa_educativo -> (nombre, abreviatura, id_unidad_academica)
datos['Programas_Educativos'] = {
    row[0]: {
        'nombre': row[1],
        'abreviatura': row[2],
        'id_unidad_academica': row[3]
    }
    for row in rows
}

# También puedes guardar por separado si lo necesitas
datos['Nombre_Programa'] = {row[0]: row[1] for row in rows}
datos['Abreviatura_Programa'] = {row[0]: row[2] for row in rows}
datos['Unidad_Academica_Programa'] = {row[0]: row[3] for row in rows}


In [38]:
aulas_param = datos['Aulas']  # lista de tuplas (id_aula, id_edificio)

query = """
SELECT A.id_aula, A.id_edificio, M.id_materia,
       CASE WHEN A.id_tipo_clase = M.id_tipo_clase THEN 1 ELSE 0 END AS compatible,
       M.id_tipo_clase AS tipo_materia,
       A.id_tipo_clase AS tipo_aula
FROM Aula A
CROSS JOIN Materia M
WHERE (A.id_aula, A.id_edificio) = ANY(%s) AND M.id_materia = ANY(%s)
"""

cursor.execute(query, (aulas_param, datos['id_Materias']))

datos['Compatibiliadad_Aula_Materia'] = {
    ((row[0], row[1]), row[2]): {
        'compatible': row[3],
        'tipo_materia': row[4],
        'tipo_aula': row[5]
    }
    for row in cursor.fetchall()
}


In [39]:
from collections import defaultdict
import unicodedata
import re

def normalizar_nombre(nombre):
    nombre = nombre.lower()
    nombre = ''.join(
        c for c in unicodedata.normalize('NFD', nombre)
        if unicodedata.category(c) != 'Mn'
    )
    nombre = nombre.strip()
    nombre = re.sub(r'\s+', ' ', nombre)
    return nombre

datos['Materias_por_Profesor'] = defaultdict(list)
datos['Profesores_por_Materia'] = defaultdict(list)
datos['Profesores_Imaginarios'] = []
datos['Profesores_Totales'] = []

# Crear dict normalizado nombre -> lista ids materias
materias_por_nombre = defaultdict(list)
for mid in datos['id_Materias']:
    nombre_original = datos['nombres_materia'][mid]
    nombre_norm = normalizar_nombre(nombre_original)
    materias_por_nombre[nombre_norm].append(mid)

cursor.execute("""
    SELECT id_profesor, id_materia 
    FROM profesor_materia
    WHERE id_profesor = ANY(%s) AND id_materia = ANY(%s)
""", (datos['Id_Profesores'], datos['id_Materias']))

for profe_id, materia_id in cursor.fetchall():
    nombre_original = datos['nombres_materia'][materia_id]
    nombre_norm = normalizar_nombre(nombre_original)
    materias_similares = materias_por_nombre[nombre_norm]
    for mid in materias_similares:
        if mid not in datos['Materias_por_Profesor'][profe_id]:
            datos['Materias_por_Profesor'][profe_id].append(mid)
        if profe_id not in datos['Profesores_por_Materia'][mid]:
            datos['Profesores_por_Materia'][mid].append(profe_id)

materias_sin_profesor = [m for m in datos['id_Materias'] if m not in datos['Profesores_por_Materia']]

profesores_imaginarios = [f"PA_{i}" for i in range(len(materias_sin_profesor))]
datos['Profesores_Imaginarios'] = profesores_imaginarios
print("Se han creado", len(profesores_imaginarios), "profesores imaginarios para las materias sin profesor asignado.")

for i, materia_id in enumerate(materias_sin_profesor):
    profe_imag = profesores_imaginarios[i]
    datos['Profesores_por_Materia'][materia_id].append(profe_imag)
    datos['Materias_por_Profesor'][profe_imag].append(materia_id)
    datos['Tipo_Contrato'][profe_imag] = 4

datos['Profesores_Totales'].extend(datos['Id_Profesores'])
datos['Profesores_Totales'].extend(profesores_imaginarios)

datos['nombres_profesor'].update({
    p: f"ProfFicticio_{i}" for i, p in enumerate(profesores_imaginarios)
})


Se han creado 0 profesores imaginarios para las materias sin profesor asignado.


In [40]:
import pandas as pd

def leer_bloques_desde_excel(ruta_archivo):
    # Leer archivo CSV
    df = pd.read_csv(ruta_archivo)

    print("Columnas en el archivo:", df.columns.tolist())

    bloques = {}

    for _, fila in df.iterrows():
        programa = fila['Programa']
        materia = fila['Materia']
        bloque = fila['Bloque']
        cupo = fila['Cupo']

        if bloque not in bloques:
            bloques[bloque] = {
                'programa_educativo': programa,
                'materias': [],
                'cupo': cupo
            }
        bloques[bloque]['materias'].append(materia)

    return bloques

ruta = r"C:\Users\bdgae\Desktop\Nueva carpeta\exportado.csv"
datos['Bloques'] = leer_bloques_desde_excel(ruta)


Columnas en el archivo: ['Programa', 'Materia', 'Bloque', 'Cupo']


In [41]:
def reemplazar_materias_por_id_en_bloques(bloques, datos):
    # Crear un dict normalizado nombre + programa -> lista de ids
    materias_por_nombre_programa = defaultdict(list)
    
    for mid in datos['id_Materias']:
        nombre_original = datos['nombres_materia'][mid]
        programa_id = datos['Programa_Educativo_Materia'][mid]
        # Normalizar nombre de materia
        nombre_norm = normalizar_nombre(nombre_original)
        materias_por_nombre_programa[(nombre_norm, programa_id)].append(mid)
    
    bloques_con_ids = {}
    for bloque, info in bloques.items():
        programa_nombre = info['programa_educativo']
        
        # Obtener id del programa educativo basado en la abreviatura o nombre
        programa_id = None
        for pid, pinfo in datos['Programas_Educativos'].items():
            # Puedes comparar por abreviatura o nombre, ajusta según tus datos
            if 'abreviatura' in pinfo and pinfo['abreviatura'] == programa_nombre:
                programa_id = pid
                break
            elif 'nombre' in pinfo and pinfo['nombre'] == programa_nombre:
                programa_id = pid
                break
        
        if programa_id is None:
            print(f"Programa educativo '{programa_nombre}' no encontrado en datos.")
            continue
        
        materias_ids = []
        for materia_nombre in info['materias']:
            materia_norm = normalizar_nombre(materia_nombre)
            posibles_ids = materias_por_nombre_programa.get((materia_norm, programa_id), [])
            if posibles_ids:
                # Si hay más de uno, toma el primero (o ajusta si quieres otro criterio)
                materias_ids.append(posibles_ids[0])
            else:
                # Materia no encontrada o no corresponde al programa
                print(f"Materia '{materia_nombre}' no encontrada para programa '{programa_nombre}', se omitirá.")
        
        bloques_con_ids[bloque] = {
            'programa_educativo': programa_nombre,
            'materias': materias_ids,
            'cupo': info['cupo']
        }
    
    return bloques_con_ids

# Uso
datos['Bloques'] = reemplazar_materias_por_id_en_bloques(datos['Bloques'], datos)


Materia 'Introducción a la FGU' no encontrada para programa 'ARQ', se omitirá.
Materia 'Inglés I' no encontrada para programa 'ARQ', se omitirá.
Materia 'Arquitectura de la Antigüedad' no encontrada para programa 'ARQ', se omitirá.
Materia 'Métodos y Estrat. Proyectuales' no encontrada para programa 'ARQ', se omitirá.
Materia 'P. Inclusión Soc.en el Espacio' no encontrada para programa 'ARQ', se omitirá.
Materia 'Dibujo II' no encontrada para programa 'ARQ', se omitirá.
Materia 'Taller de Diseño Integral I' no encontrada para programa 'ARQ', se omitirá.
Materia 'Matem. Aplicadas a la Arqui.II' no encontrada para programa 'ARQ', se omitirá.
Materia 'Introducc. a las Instalaciones' no encontrada para programa 'ARQ', se omitirá.
Materia 'Conc. Básicos de Construcción' no encontrada para programa 'ARQ', se omitirá.
Materia 'Inglés II' no encontrada para programa 'ARQ', se omitirá.
Materia 'Introducción a la FGU' no encontrada para programa 'ARQ', se omitirá.
Materia 'Inglés I' no encontrad

In [42]:
# Imprimir todas las claves (columnas) del diccionario 'datos' y el primer elemento de cada una (si aplica)
print("Columnas (claves) en 'datos':")
for key in datos.keys():
    print(f"{key}: ", end="")
    valor = datos[key]
    # Si es lista y no está vacía, imprime el primer elemento
    if isinstance(valor, list) and len(valor) > 0:
        print(valor[0])
    # Si es dict y no está vacío, imprime el primer par clave-valor
    elif isinstance(valor, dict) and len(valor) > 0:
        primera_clave = next(iter(valor))
        print(f"{primera_clave}: {valor[primera_clave]}")
    # Si es defaultdict y no está vacío, imprime el primer par clave-valor
    elif "defaultdict" in str(type(valor)) and len(valor) > 0:
        primera_clave = next(iter(valor))
        print(f"{primera_clave}: {valor[primera_clave]}")
    else:
        print(valor)

Columnas (claves) en 'datos':
Id_Profesores: 100002999
nombres_profesor: 100002999: GARCIA - ZENTENO EDUARDO
Tipo_Contrato: 100002999: 3
Detalles_Contratos: 1: {'min': 12, 'max': 20}
id_Materias: 1
nombres_materia: 1: Teoría de la Arquitectura
Horas_Semana_Materia: 1: 3
Tipo_Clase: 1: 4
Programa_Educativo_Materia: 1: 5
Aulas: (101, 2)
Tipo_Aula: (101, 2): 3
Id_Edificios: 1
Edificios: 1: EMA1
Dia: 1
Hora: 1
Id_Programas_Educativos: 1
Programas_Educativos: 1: {'nombre': 'Licenciatura en Biología', 'abreviatura': 'BIO', 'id_unidad_academica': 1}
Nombre_Programa: 1: Licenciatura en Biología
Abreviatura_Programa: 1: BIO
Unidad_Academica_Programa: 1: 1
Compatibiliadad_Aula_Materia: ((101, 2), 1): {'compatible': 0, 'tipo_materia': 4, 'tipo_aula': 3}
Materias_por_Profesor: 100004288: [1, 2, 3]
Profesores_por_Materia: 1: [100004288, 100020344, 100502922, 100518251, 100528894, 100529762, 100538141]
Profesores_Imaginarios: []
Profesores_Totales: 100002999
Bloques: ARQBLO1: {'programa_educativo': 

In [43]:
# Definir los conjuntos
model = pyo.ConcreteModel()

# Conjuntos

materias_bloque_expandidas = []

for bloque, info in datos['Bloques'].items():
    for materia in info['materias']:
        materias_bloque_expandidas.append((bloque, materia))


model.Profesores = pyo.Set(initialize=datos['Profesores_Totales'])
model.MateriasPorBloqueExpandido = pyo.Set(initialize=materias_bloque_expandidas, dimen=2)
model.Materias = pyo.Set(initialize=sorted(set(m for _, m in materias_bloque_expandidas)))
model.Aulas = pyo.Set(dimen=2, initialize=[(aula_id, edificio_id) for aula_id, edificio_id in datos['Aulas']])
model.BloquesHorarios = pyo.Set(dimen=2, initialize=[(dia, hora) for dia in datos['Dia'] for hora in datos['Hora']])
model.Dias = pyo.Set(initialize=datos['Dia'])
model.Hora = pyo.Set(initialize=datos['Hora'])



bloques_con_materias = {bloque: info for bloque, info in datos['Bloques'].items() if info.get('materias')}

# Conjunto de bloques válidos
model.Bloques = pyo.Set(initialize=bloques_con_materias.keys())

# Conjunto MateriasPorBloque (solo para bloques con materias)
model.MateriasPorBloque = pyo.Set(
    model.Bloques,
    initialize={
        bloque: sorted(info['materias']) for bloque, info in bloques_con_materias.items()
    }
)


In [44]:
# Parámetros

model.Horas_Semana_Materia = pyo.Param(
    model.MateriasPorBloqueExpandido,
    initialize={(bloque, m): datos['Horas_Semana_Materia'][m] for (bloque, m) in materias_bloque_expandidas}
)
model.Tipo_Clase = pyo.Param(model.Materias, initialize=datos['Tipo_Clase'])
model.Tipo_Aula = pyo.Param(model.Aulas, initialize=datos['Tipo_Aula'])
model.Compatibilidad_Aula_Materia = pyo.Param(model.Aulas, model.Materias, initialize={
    (aula, materia): datos['Compatibiliadad_Aula_Materia'].get((aula, materia), {}).get('compatible', 0)
    for aula in model.Aulas for materia in model.Materias
})  # Compatibilidad entre aulas y materias (0 o 1)
# Relación entre profesores y las materias que pueden enseñar
model.Relacion_Profesor_Materia = pyo.Param(
    model.Profesores, model.MateriasPorBloqueExpandido,
    initialize={
        (profe_id, (bloque, materia_id)): 1 if materia_id in datos['Materias_por_Profesor'][profe_id] else 0
        for profe_id in model.Profesores
        for (bloque, materia_id) in materias_bloque_expandidas
    }
)






In [45]:
# Variables

X_index = [
    (p, bloque, m, a, bh)
    for p in datos['Profesores_Totales']
    for (bloque, m) in materias_bloque_expandidas
    if m in datos['Materias_por_Profesor'].get(p, [])
    for a in datos['Aulas']
    if datos['Compatibiliadad_Aula_Materia'].get((a, m), {}).get('compatible', 0) == 1
    for bh in [(d, h) for d in datos['Dia'] for h in datos['Hora']]
]


PROFESOR = 0
BLOQUE = 1
MATERIA = 2
AULA = 3
EDIFICIO = 4
DIA = 5
HORA = 6

# X_Index 
#[(100004288, 'ARQBLO1', 1, (102, 5), (1, 1))
# Model.X tiene la siguiente estructura para sus indices, pyomo aplana las tuplas :
# (100004288, ARQBLO1, 1, 102, 5, 1, 1) 

model.X = pyo.Var(X_index, domain=pyo.Binary)

# Variables generales: profesor puede enseñar materia (relación) - incluye imaginarios
Y_index = [(p, m) for p in datos['Profesores_Totales'] for m in datos['Materias_por_Profesor'].get(p, [])]
model.Y = pyo.Var(Y_index, domain=pyo.Binary)

# Variables generales: profesor tiene asignada la materia
Y_hat_index = [(p, bloque, m) for p in datos['Profesores_Totales'] for (bloque, m) in materias_bloque_expandidas]
model.Y_hat = pyo.Var(Y_hat_index, domain=pyo.Binary)


# Variables generales: aula asignada a materia (sin cambios)
Z_index = [(aula, edificio, materia) 
           for ((aula, edificio), materia), val in datos['Compatibiliadad_Aula_Materia'].items() 
           if val['compatible'] == 1]
model.Z = pyo.Var(Z_index, domain=pyo.Binary)

# Variables generales: materia impartida en dia d
Z_hat_index = [(bloque, m, d) for (bloque, m) in materias_bloque_expandidas for d in datos['Dia']]
model.Z_hat = pyo.Var(Z_hat_index, domain=pyo.Binary)


# # Variable Materia X es dada en hora H el dia D
S_Index = [(bloque, m, h, d) for (bloque, m) in materias_bloque_expandidas for h in datos['Hora'] for d in datos['Dia']]
model.S = pyo.Var(S_Index, domain=pyo.Binary)

model.U = pyo.Var(
    [(aula, edificio, bloque, materia)
     for (aula, edificio) in model.Aulas
     for (bloque, materia) in model.MateriasPorBloqueExpandido],
    domain=pyo.Binary
)





print(f"Variables reducidas de {len(datos['Profesores_Totales'])*len(datos['id_Materias'])*len(datos['Aulas'])*len(datos['Dia'])*len(datos['Hora'])} a {len(X_index)}")

Variables reducidas de 22932000 a 1899450


In [ ]:
def vincular_X_con_U_rule(model, p, bloque, m, aula, edificio, dia, hora):
    return model.X[p, bloque, m, aula, edificio, dia, hora] <= model.U[aula, edificio, bloque, m]

model.Vincular_X_con_U = pyo.Constraint(X_index, rule=vincular_X_con_U_rule)


def ubicacion_unica_para_bloque_materia_rule(model, bloque, m):
    return sum(model.U[aula, edificio, bloque, m] for (aula, edificio) in model.Aulas) == 1

model.Ubicacion_Unica_BloqueMateria = pyo.Constraint(model.MateriasPorBloqueExpandido, rule=ubicacion_unica_para_bloque_materia_rule)


In [47]:
# def no_solapamiento_en_bloque_rule(model, bloque, dia, hora):
#     # Suma todas las materias de ese bloque en ese día y hora
#     return sum(model.S[bloque, m, hora, dia] for m in model.MateriasPorBloque[bloque]) <= 1

# model.NoSolapamientoEnBloque = pyo.Constraint(
#     model.Bloques,
#     model.Dias,
#     model.Hora,
#     rule=no_solapamiento_en_bloque_rule
# )


In [48]:
# Eliminar versión anterior si existe
if hasattr(model, 'Restriccion_Materia_Tiene_Profe'):
    model.del_component(model.Restriccion_Materia_Tiene_Profe)

# Regla adaptada
def materia_tiene_profe_rule(model, bloque, materia):
    return sum(
        model.Y_hat[p, bloque, materia] * model.Relacion_Profesor_Materia[p, (bloque, materia)]
        for p in model.Profesores
    ) == 1

# Restricción sobre MateriasPorBloqueExpandido
model.Restriccion_Materia_Tiene_Profe = pyo.Constraint(
    model.MateriasPorBloqueExpandido,
    rule=materia_tiene_profe_rule
)


In [49]:
# Construir el conjunto plano (lista) de (profesor, bloque, materia) válidos
tripletes = []
for p in model.Profesores:
    for bloque in model.Bloques:
        for m in model.MateriasPorBloque[bloque]:
            tripletes.append((p, bloque, m))

# Crear un Set en el modelo con esos índices
model.IndicesHorasSemanales = pyo.Set(initialize=tripletes, dimen=3)

# Construir diccionario para acceder rápido a los índices de X
indices_X_por_tripleta = {}
for idx in model.X.index_set():
    p = idx[PROFESOR]
    bloque = idx[BLOQUE]
    m = idx[MATERIA]
    indices_X_por_tripleta.setdefault((p, bloque, m), []).append(idx)

# Función regla que ahora toma solo 3 argumentos (p, bloque, m)
def horas_semana_materia_rule(model, p, bloque, m):
    indices = indices_X_por_tripleta.get((p, bloque, m), [])
    suma = sum(model.X[idx] for idx in indices)
    return suma == datos['Horas_Semana_Materia'][m] * model.Y_hat[p, bloque, m]

# Eliminar restricción si ya existe
if hasattr(model, 'Restriccion_Horas_Semanales'):
    model.del_component(model.Restriccion_Horas_Semanales)

# Crear la restricción usando el conjunto plano
model.Restriccion_Horas_Semanales = pyo.Constraint(
    model.IndicesHorasSemanales, rule=horas_semana_materia_rule
)


In [50]:
#Funciona No solapamiento de aulas
if hasattr(model, 'Restriccion_Aula_Unica'):
    model.del_component(model.Restriccion_Aula_Unica)

# Precomputar las combinaciones de aula, edificio, día y hora
solapamientos = {}

for idx in model.X:
    aula = idx[AULA]
    edificio = idx[EDIFICIO]
    dia = idx[DIA]
    hora = idx[HORA]
    
    if (aula, edificio, dia, hora) not in solapamientos:
        solapamientos[(aula, edificio, dia, hora)] = []

    solapamientos[(aula, edificio, dia, hora)].append(model.X[idx])

# Restricción para el no solapamiento de aulas
def no_solapamiento_aula_rule(model, aula_id, edificio_id, dia, hora):
    if (aula_id, edificio_id, dia, hora) not in solapamientos:
        return pyo.Constraint.Skip  # Si no existe ninguna asignación para esa combinación

    exprs = solapamientos[(aula_id, edificio_id, dia, hora)]
    return sum(exprs) <= 1

# Aplicamos la restricción optimizada
model.Restriccion_Aula_Unica = pyo.Constraint(
    model.Aulas, model.BloquesHorarios,
    rule=no_solapamiento_aula_rule
)




In [51]:
# # Precalcular los índices por profesor
# indices_por_profesor = {p: [] for p in model.Profesores}

# for idx in model.X:
#     profesor = idx[PROFESOR]
#     if profesor in indices_por_profesor:
#         indices_por_profesor[profesor].append(idx)

# # Eliminar restricciones si ya existen
# if hasattr(model, 'Restriccion_Max_Horas_Profesor'):
#     model.del_component(model.Restriccion_Max_Horas_Profesor)

# tipo_contrato_por_profesor = datos['Tipo_Contrato']
# detalles_contratos = datos['Detalles_Contratos']

# # Restricción máxima horas por profesor
# def max_horas_profesor_rule(model, p):
#     if (
#         p not in tipo_contrato_por_profesor
#         or p not in indices_por_profesor
#         or not indices_por_profesor[p]  # Lista vacía
#     ):
#         return pyo.Constraint.Skip

#     contrato_id = tipo_contrato_por_profesor[p]
#     max_horas = detalles_contratos[contrato_id]['max']

#     # Sumar las variables X asignadas a ese profesor (cada X representa 1 hora en un bloque/materia/aula/...)
#     return sum(model.X[idx] for idx in indices_por_profesor[p]) <= max_horas

# model.Restriccion_Max_Horas_Profesor = pyo.Constraint(model.Profesores, rule=max_horas_profesor_rule)


In [52]:
# Eliminar restricción si ya existe
if hasattr(model, 'Restriccion_Ubicacion_Unica_Materia'):
    model.del_component(model.Restriccion_Ubicacion_Unica_Materia)

def ubicacion_unica_por_materia_rule(model, m):
    # Suma de todas las asignaciones Z para la materia m, sobre todas aulas y edificios
    return sum(model.Z[aula, edificio, m] for (aula, edificio, mm) in model.Z.index_set() if mm == m) == 1

model.Restriccion_Ubicacion_Unica_Materia = pyo.Constraint(
    model.Materias, rule=ubicacion_unica_por_materia_rule
)


In [53]:
# Eliminar restricción si ya existe
if hasattr(model, 'Restriccion_Profesor_Unico'):
    model.del_component(model.Restriccion_Profesor_Unico)

# Precomputar las asignaciones por profesor, día y hora
solapamientos_profesor = {}

for idx in model.X:
    profesor = idx[PROFESOR]
    dia = idx[DIA]
    hora = idx[HORA]

    if (profesor, dia, hora) not in solapamientos_profesor:
        solapamientos_profesor[(profesor, dia, hora)] = []

    solapamientos_profesor[(profesor, dia, hora)].append(model.X[idx])

# Restricción que limita al profesor a no impartir más de una materia simultáneamente
def no_solapamiento_profesor_rule(model, p, d, h):
    if (p, d, h) not in solapamientos_profesor:
        return pyo.Constraint.Skip
    return sum(solapamientos_profesor[(p, d, h)]) <= 1

model.Restriccion_Profesor_Unico = pyo.Constraint(
    model.Profesores, model.Dias, model.Hora,
    rule=no_solapamiento_profesor_rule
)


In [54]:
# # Eliminar restricción si existe
# if hasattr(model, 'Restriccion_Profesor_Compatible'):
#     model.del_component(model.Restriccion_Profesor_Compatible)

# # Precalcular variables X que son incompatibles (Relacion_Profesor_Materia == 0)
# x_incompatibles_profesor = [
#     model.X[idx] for idx in model.X.index_set()
#     if (idx[0], idx[1], idx[2]) not in model.Relacion_Profesor_Materia or model.Relacion_Profesor_Materia[idx[0], idx[1], idx[2]] == 0
# ]

# # Restricción: esas X deben ser 0
# def profesor_compatible_rule(model, i):
#     return x_incompatibles_profesor[i] == 0

# model.Restriccion_Profesor_Compatible = pyo.Constraint(
#     range(len(x_incompatibles_profesor)),
#     rule=profesor_compatible_rule
# )


In [55]:
# # Eliminar restricción si existe
# if hasattr(model, 'Restriccion_Aula_Compatible'):
#     model.del_component(model.Restriccion_Aula_Compatible)

# # Precalcular X que tengan aula incompatible con la materia
# x_incompatibles_aula = [
#     model.X[idx] for idx in model.X.index_set()
#     if (idx[3], idx[4], idx[2]) not in model.Compatibilidad_Aula_Materia or model.Compatibilidad_Aula_Materia[idx[3], idx[4], idx[2]] == 0
# ]

# # Restricción: esas X deben ser 0
# def aula_compatible_rule(model, i):
#     return x_incompatibles_aula[i] == 0

# model.Restriccion_Aula_Compatible = pyo.Constraint(
#     range(len(x_incompatibles_aula)),
#     rule=aula_compatible_rule
# )


In [56]:
# # Eliminar restricciones si existen
# if hasattr(model, 'Relacion_X_Z_1'):
#     model.del_component(model.Relacion_X_Z_1)
# if hasattr(model, 'Relacion_X_Z_2'):
#     model.del_component(model.Relacion_X_Z_2)

# X_index = model.X.index_set()

# def relacion_X_Z_rule_1(model, p, b, m, aula, edificio, dia, hora):
#     # Acceso directo con control de existencia para evitar error
#     if (aula, edificio, m) in model.Z:
#         return model.X[p, b, m, aula, edificio, dia, hora] <= model.Z[aula, edificio, m]
#     else:
#         # Si no existe Z para esta combinación, no permitir X=1
#         return model.X[p, b, m, aula, edificio, dia, hora] == 0

# def relacion_X_Z_rule_2(model, p, b, m, aula, edificio, dia, hora):
#     if (aula, edificio, m) in model.Z:
#         return model.Z[aula, edificio, m] >= model.X[p, b, m, aula, edificio, dia, hora]
#     else:
#         # Si no existe Z, no permitir asignación
#         return model.X[p, b, m, aula, edificio, dia, hora] == 0

# model.Relacion_X_Z_1 = pyo.Constraint(X_index, rule=relacion_X_Z_rule_1)
# model.Relacion_X_Z_2 = pyo.Constraint(X_index, rule=relacion_X_Z_rule_2)


In [57]:

# # --- Restricción: Las clases de una materia en un día deben ser consecutivas ---

# # Eliminar componente anterior si ya existe
# if hasattr(model, 'Restriccion_Consecutividad_Materia_Dia'):
#     model.del_component(model.Restriccion_Consecutividad_Materia_Dia)

# # Precalcular clases posibles por (materia, día, hora)
# clases_por_materia_dia_hora = {}

# for (p, b, m, aula, edificio, d, h) in model.X.index_set():
#     key = (m, d, h)
#     if key not in clases_por_materia_dia_hora:
#         clases_por_materia_dia_hora[key] = []
#     clases_por_materia_dia_hora[key].append(model.X[p, b, m, aula, edificio, d, h])

# # Definir el conjunto de índices válidos (no incluye la última hora del día)
# consecutividad_index = [
#     (m, d, h)
#     for m in datos['id_Materias']
#     for d in datos['Dia']
#     for h in datos['Hora'][:-1]  # Excluimos la última hora para h+1
# ]

# # Regla: si hay clase en h+1, debe haber clase en h
# def consecutividad_materia_dia_rule(model, m, d, h):
#     clases_h = clases_por_materia_dia_hora.get((m, d, h), [])
#     clases_h1 = clases_por_materia_dia_hora.get((m, d, h + 1), [])

#     # Si no hay clases posibles en h+1, saltar restricción
#     if not clases_h1:
#         return pyo.Constraint.Skip

#     return sum(clases_h1) <= sum(clases_h)

# # Crear restricción en el modelo
# model.Restriccion_Consecutividad_Materia_Dia = pyo.Constraint(
#     consecutividad_index,
#     rule=consecutividad_materia_dia_rule
# )


In [58]:
# # ==============================
# # Restricción: No solapamiento entre materias del mismo bloque
# # ==============================

# # Precalcular combinaciones únicas de materias por bloque (sin repetir ni reflejar pares)
# materias_mismo_bloque_sin_repetir = []

# for bloque in model.Bloques:
#     materias = list(model.MateriasPorBloque[bloque])
#     for i in range(len(materias)):
#         for j in range(i + 1, len(materias)):
#             m1, m2 = materias[i], materias[j]
#             materias_mismo_bloque_sin_repetir.append((bloque, m1, m2))

# # Crear restricción con base en las combinaciones precalculadas
# model.NoSolapamientoMateriasEnMismoBloque = pyo.Constraint(
#     [
#         (bloque, m1, m2, d, h)
#         for (bloque, m1, m2) in materias_mismo_bloque_sin_repetir
#         for d in datos['Dia']
#         for h in datos['Hora']
#     ],
#     rule=lambda model, bloque, m1, m2, d, h:
#         model.S[bloque, m1, h, d] + model.S[bloque, m2, h, d] <= 1
# )


In [59]:
if hasattr(model, 'objetivo_dummy'):
    model.del_component(model.objetivo_dummy)

model.objetivo_dummy = pyo.Objective(expr=1, sense=pyo.minimize)

In [60]:
# Obtener todas las restricciones activas del modelo y almacenarlas en una lista
restricciones = []

print("Restricciones definidas en el modelo:")
for nombre, componente in model.component_map(pyo.Constraint, active=True).items():
    restricciones.append(nombre)
    print(f"- {nombre}")

# Ahora tienes una lista con los nombres de las restricciones
# Puedes usarla después si necesitas procesarlas de otra forma


Restricciones definidas en el modelo:
- Vincular_X_con_U
- Ubicacion_Unica_BloqueMateria
- Restriccion_Materia_Tiene_Profe
- Restriccion_Horas_Semanales
- Restriccion_Aula_Unica
- Restriccion_Ubicacion_Unica_Materia
- Restriccion_Profesor_Unico


In [61]:
def resolver_modelo(model, mipgap=0.02, threads=4, emphasis=1):
    from pyomo.opt import SolverFactory, TerminationCondition

    solver = SolverFactory('cplex')

    if not solver.available():
        raise RuntimeError("CPLEX no está disponible. Verifica la instalación o el PATH.")

    resultado = solver.solve(model, tee=True)

    # Verificar si la solución fue óptima o factible
    condicion = resultado.solver.termination_condition
    if condicion in [TerminationCondition.optimal, TerminationCondition.feasible]:
        print(f"Solución encontrada: {condicion}")
    else:
        print(f"Problema durante la resolución: {condicion}")
        print("Revisa el modelo o ajusta parámetros del solver.")

    return resultado


In [62]:
import pandas as pd
from collections import defaultdict
import pyomo.environ as pyo  # Asegúrate de tener esta importación si usas Pyomo

def mostrar_bloque(dia, hora_inicio, hora_fin):
    dias = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes']
    dia_str = dias[dia - 1] if 1 <= dia <= 5 else f"Día{dia}"
    hora_inicio_real = 7 + hora_inicio
    hora_fin_real = 7 + hora_fin
    return f"{dia_str} {hora_inicio_real:02d}:00 - {hora_fin_real:02d}:00"

def mostrar_asignacion(model, datos):
    sesiones = defaultdict(list)

    # Agrupar sesiones por clave común (profesor, bloque, materia, aula, edificio, día)
    for idx in model.X:
        if pyo.value(model.X[idx]) > 0.5:
            p, bloque, m, aula, edificio_id, dia, hora = idx
            sesiones[(p, bloque, m, aula, edificio_id, dia)].append(hora)

    agrupadas = defaultdict(list)

    # Agrupar horarios consecutivos por sesión
    for (p, bloque, m, aula, edificio_id, dia), horas in sesiones.items():
        horas = sorted(horas)
        bloques_horarios = []
        inicio = fin = horas[0]

        for h in horas[1:]:
            if h == fin + 1:
                fin = h
            else:
                bloques_horarios.append((inicio, fin + 1))
                inicio = fin = h
        bloques_horarios.append((inicio, fin + 1))  # último bloque

        for hora_inicio, hora_fin in bloques_horarios:
            horario_str = mostrar_bloque(dia, hora_inicio, hora_fin)
            agrupadas[(p, bloque, m, aula, edificio_id)].append(horario_str)

    asignaciones = []

    print("\nAsignaciones de profesores, materias, aulas y horarios:")
    for (p, bloque, m, aula, edificio_id), lista_horarios in agrupadas.items():
        profesor_nombre = datos.get('nombres_profesor', {}).get(p, str(p))
        materia_nombre = datos.get('nombres_materia', {}).get(m, str(m))
        edificio_nombre = datos.get('Edificios', {}).get(edificio_id, f"Edificio {edificio_id}")
        programa_educativo = datos.get('Bloques', {}).get(bloque, {}).get('programa_educativo', 'Desconocido')

        horarios_combinados = "; ".join(lista_horarios)

        print(f"Profesor: {profesor_nombre} | Materia: {materia_nombre} | Aula: {aula} "
              f"(Edificio {edificio_nombre}) | Horarios: {horarios_combinados} | "
              f"Bloque: {bloque} | Programa: {programa_educativo}")

        asignaciones.append({
            'id_materia': m,
            'Materia': materia_nombre,
            'Bloque': bloque,
            'Programa_Educativo': programa_educativo,
            'Profesor': profesor_nombre,
            'Edificio': edificio_nombre,
            'Aula': aula,
            'Horarios': horarios_combinados
        })

    # Exportar a Excel
    df = pd.DataFrame(asignaciones, columns=[
        'id_materia',
        'Materia',
        'Bloque',
        'Programa_Educativo',
        'Profesor',
        'Edificio',
        'Aula',
        'Horarios'
    ])
    df.to_excel('asignaciones.xlsx', index=False)
    print("\nArchivo 'asignaciones.xlsx' creado con éxito.")

    return asignaciones




def mostrar_horarios_por_bloque(asignaciones):
    from collections import defaultdict

    dias_orden = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes']

    bloques = defaultdict(list)

    for asignacion in asignaciones:
        bloque = asignacion['Bloque']
        materia = asignacion['Materia']
        profesor = asignacion['Profesor']
        aula = asignacion['Aula']
        edificio = asignacion['Edificio']
        horarios = asignacion['Horarios']
        programa = asignacion['Programa_Educativo']

        # Separar por cada horario individual
        for horario in horarios.split('; '):
            dia, resto = horario.split(' ', 1)
            bloques[(bloque, programa)].append({
                'Día': dia,
                'Hora': resto,
                'Materia': materia,
                'Profesor': profesor,
                'Aula': aula,
                'Edificio': edificio
            })

    print("\n📘 Horarios agrupados por Bloque y Programa:\n")

    for (bloque, programa), asigns in sorted(bloques.items()):
        print("=" * 70)
        print(f"🔹 BLOQUE: {bloque} | PROGRAMA EDUCATIVO: {programa}")
        print("=" * 70)

        # Ordenar por día y hora
        asigns.sort(key=lambda x: (dias_orden.index(x['Día']), x['Hora']))

        # Cabecera estilo tabla
        print(f"{'Día':<12} {'Hora':<15} {'Materia':<35} {'Profesor':<30} {'Aula':<10} {'Edificio'}")
        print("-" * 130)

        for a in asigns:
            print(f"{a['Día']:<12} {a['Hora']:<15} {a['Materia'][:34]:<35} "
                  f"{a['Profesor'][:29]:<30} {a['Aula']:<10} {a['Edificio']}")
        print()  # Espacio entre bloques

    # Exportar a Excel (ordenado también)
    filas = []
    for (bloque, programa), asigns in bloques.items():
        for a in asigns:
            filas.append({
                'Bloque': bloque,
                'Programa_Educativo': programa,
                'Día': a['Día'],
                'Hora': a['Hora'],
                'Materia': a['Materia'],
                'Profesor': a['Profesor'],
                'Aula': a['Aula'],
                'Edificio': a['Edificio']
            })

    df_bloques = pd.DataFrame(filas)
    df_bloques['Día'] = pd.Categorical(df_bloques['Día'], categories=dias_orden, ordered=True)
    df_bloques.sort_values(['Bloque', 'Programa_Educativo', 'Día', 'Hora'], inplace=True)
    df_bloques.to_excel('horarios_por_bloque.xlsx', index=False)
    print("📁 Archivo 'horarios_por_bloque.xlsx' creado con éxito.\n")


In [63]:
# Resolver el modelo
resultados = resolver_modelo(model)

# Verificar si encontró solución
if (resultados.solver.status == pyo.SolverStatus.ok and
    resultados.solver.termination_condition == pyo.TerminationCondition.optimal):

    print("Solución óptima encontrada.\n")
    asignaciones_lista = mostrar_asignacion(model,datos)
    mostrar_horarios_por_bloque(asignaciones_lista)


    import pandas as pd
    df = pd.DataFrame(asignaciones_lista)
    profesores_usados = {a['Profesor'] for a in asignaciones_lista}
    total_usados = len(profesores_usados)
    total_profesores = datos['Profesores_Totales']

    print(f"\nProfesores asignados: {total_usados} de {len(total_profesores)}")

    # Expandir horarios por fila para detectar solapes
    df_exploded = df.copy()
    df_exploded['Horario'] = df_exploded['Horarios'].str.split('; ')
    df_exploded = df_exploded.explode('Horario')

    # Verificar solapes
    solapes_profesor = df_exploded[df_exploded.duplicated(['Profesor', 'Horario'], keep=False)]
    solapes_aula = df_exploded[df_exploded.duplicated(['Aula', 'Edificio', 'Horario'], keep=False)]

    if solapes_profesor.empty:
        print("✅ No hay solapamientos de profesor.")
    else:
        print("⚠️ Hay solapamientos de profesor:")
        print(solapes_profesor[['Profesor', 'Materia', 'Horario']].drop_duplicates())

    if solapes_aula.empty:
        print("✅ No hay solapamientos de aula.")
    else:
        print("⚠️ Hay solapamientos de aula:")
        print(solapes_aula[['Aula', 'Edificio', 'Materia', 'Horario']].drop_duplicates())

else:
    print("No se encontró solución óptima.")
    print("Estado:", resultados.solver.status)
    print("Condición de terminación:", resultados.solver.termination_condition)



Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 22.1.1.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2022.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile 'C:\Users\bdgae\AppData\Local\Temp\tmphlokh4zw.cplex.log' open.
CPLEX> Problem 'C:\Users\bdgae\AppData\Local\Temp\tmpfdfpbchy.pyomo.lp' read.
Read time = 11.00 sec. (304.87 ticks)
CPLEX> Problem name         : C:\Users\bdgae\AppData\Local\Temp\tmpfdfpbchy.pyomo.lp
Objective sense      : Minimize
Variables            : 1946440  [Fix: 1,  Binary: 1946439]
Objective nonzeros   :       1
Linear constraints   : 1940314  [Less: 1904450,  Equal: 35864]
  Nonzeros           : 9544806
  RHS nonzeros       :    5192

Variables            : Min LB: 0.000000         Max UB: 1.000000       
Objective nonzer

In [64]:
import re

def revisar_restricciones(df):
    errores = []

    # ⚠️ Expandir horarios múltiples en filas individuales
    df_exp = df.copy()
    df_exp['Horario'] = df_exp['Horarios'].str.split('; ')
    df_exp = df_exp.explode('Horario')

    # Asegurar que 'Día' exista
    df_exp['Día'] = df_exp['Horario'].apply(lambda h: h.split()[0] if isinstance(h, str) else None)

    # Extraer hora de inicio
    def extraer_hora_inicio(horario):
        match = re.search(r'(\d{2}):\d{2}', horario)
        return int(match.group(1)) if match else None

    df_exp['Hora_inicio'] = df_exp['Horario'].apply(extraer_hora_inicio)

    # Clave principal para agrupación
    clave_materia = ['id_materia', 'Programa_Educativo']

    # 1. Materias sin profesor
    sin_profe = df_exp[df_exp['Profesor'].isnull()][clave_materia].drop_duplicates()
    if not sin_profe.empty:
        errores.append(f"Materias sin profesor asignado:\n{sin_profe.to_dict(orient='records')}")

    # 2. Materias con menos de 1 horario
    horas_por_materia = df_exp.groupby(clave_materia)['Horario'].nunique()
    pocas_horas = horas_por_materia[horas_por_materia < 1]
    if not pocas_horas.empty:
        errores.append(f"Materias con menos horas semanales:\n{pocas_horas.index.tolist()}")

    # 3. Materias en múltiples aulas
    aulas_por_materia = df_exp.groupby(clave_materia)['Aula'].nunique()
    multiples_aulas = aulas_por_materia[aulas_por_materia > 1]
    if not multiples_aulas.empty:
        errores.append(f"Materias en múltiples aulas:\n{multiples_aulas.index.tolist()}")

    # 4. Profesores con más de 10 horarios
    max_horas = 10
    horas_por_profesor = df_exp.groupby('Profesor')['Horario'].nunique()
    excedidos = horas_por_profesor[horas_por_profesor > max_horas]
    if not excedidos.empty:
        errores.append(f"Profesores con más de {max_horas} horarios:\n{excedidos.index.tolist()}")

    # 5. Materias en múltiples edificios
    edificios_por_materia = df_exp.groupby(clave_materia)['Edificio'].nunique()
    multiples_edificios = edificios_por_materia[edificios_por_materia > 1]
    if not multiples_edificios.empty:
        errores.append(f"Materias en múltiples edificios:\n{multiples_edificios.index.tolist()}")

    # 6. Materias con más de un profesor
    profes_por_materia = df_exp.groupby(clave_materia)['Profesor'].nunique()
    varios_profes = profes_por_materia[profes_por_materia > 1]
    if not varios_profes.empty:
        errores.append(f"Materias con más de un profesor asignado:\n{varios_profes.index.tolist()}")

    # 7. Profesores con clases simultáneas
    profe_horario = df_exp.groupby(['Profesor', 'Horario']).size()
    conflictos_profe = profe_horario[profe_horario > 1]
    if not conflictos_profe.empty:
        errores.append(f"Profesores con clases simultáneas:\n{conflictos_profe.index.tolist()}")

    # 8. Aulas con clases simultáneas
    aula_horario = df_exp.groupby(['Aula','Edificio','Horario']).size()
    conflictos_aula = aula_horario[aula_horario > 1]
    if not conflictos_aula.empty:
        errores.append(f"Aulas con clases simultáneas:\n{conflictos_aula.index.tolist()}")

    # 11. Clases no consecutivas por materia y día
    for (m_id, programa, dia), grupo in df_exp.groupby(clave_materia + ['Día']):
        horas = sorted(grupo['Hora_inicio'].dropna().unique())
        if len(horas) > 1:
            consecutivas = all(horas[i+1] == horas[i] + 1 for i in range(len(horas)-1))
            if not consecutivas:
                errores.append(f"Clases no consecutivas para materia {m_id} (Programa {programa}) el día {dia}: {horas}")

    # Resultados
    if errores:
        print("❌ Restricciones violadas:")
        for e in errores:
            print(f"- {e}")
    else:
        print("✅ Todas las restricciones se cumplen.")

# Ejecutar verificación
revisar_restricciones(df)


❌ Restricciones violadas:
- Materias en múltiples aulas:
[(1, 'ARQ'), (2, 'ARQ'), (3, 'ARQ'), (4, 'ARQ'), (5, 'ARQ'), (6, 'ARQ'), (7, 'ARQ'), (8, 'BIO'), (9, 'BIO'), (10, 'BIO')]
- Profesores con más de 10 horarios:
['ALONSO - PEREZ CARLOS', 'CONTRERAS - ALVARADO MINERVA', 'DIAZ - DE ANDA ALFREDO', 'JAEN - VARGAS MARIA GUADALUPE', 'MALDONADO - CASTRO NAHELY', 'MALDONADO - SANCHEZ PABLO', 'MENDOZA - DIAZ LUIS FELIPE', 'MORA - RODRIGUEZ DAVID', 'MORALES - ORTEGA JOSE ALEJANDRO', 'REYES - ROMERO MARIBEL', 'ROSAS - LORANCA ROBERTO', 'SANTIBAÑEZ - AGUASCALIENTES NORMA ANGELICA', 'VAZQUEZ - GOMEZ NOEMI ZAHIRA', 'VERGARA - JUAREZ ENRIQUE', 'VIDAL - FLORES ANA MARIA DEL RAYO']
- Materias en múltiples edificios:
[(1, 'ARQ'), (2, 'ARQ'), (3, 'ARQ'), (4, 'ARQ'), (5, 'ARQ'), (6, 'ARQ'), (7, 'ARQ'), (8, 'BIO'), (9, 'BIO'), (10, 'BIO')]
- Materias con más de un profesor asignado:
[(1, 'ARQ'), (2, 'ARQ'), (3, 'ARQ'), (4, 'ARQ'), (5, 'ARQ'), (6, 'ARQ'), (7, 'ARQ'), (8, 'BIO'), (9, 'BIO'), (10, 'BIO')]

In [65]:
# Verificar si materias del mismo bloque están sobrepuestas en horarios (aula y horario)
def materias_solapadas_en_bloque(df):
    # Expandir horarios múltiples en filas individuales
    df_exp = df.copy()
    df_exp['Horario'] = df_exp['Horarios'].str.split('; ')
    df_exp = df_exp.explode('Horario')

    # Agrupar por Bloque y Horario, contar materias distintas
    solapes = (
        df_exp.groupby(['Bloque', 'Horario'])
        .agg({'id_materia': 'nunique'})
        .reset_index()
    )
    # Filtrar donde hay más de una materia en el mismo bloque y horario
    solapes = solapes[solapes['id_materia'] > 1]

    if not solapes.empty:
        print("❌ Hay materias del mismo bloque sobrepuestas en el mismo horario:")
        print(solapes)
    else:
        print("✅ No hay materias del mismo bloque sobrepuestas en horarios.")

# Usar con el dataframe de asignaciones (df)
materias_solapadas_en_bloque(df)

❌ Hay materias del mismo bloque sobrepuestas en el mismo horario:
       Bloque                  Horario  id_materia
2     ARQBLO1     Jueves 16:00 - 17:00           2
3     ARQBLO1     Jueves 17:00 - 18:00           2
17    ARQBLO1    Viernes 11:00 - 12:00           2
20    ARQBLO1    Viernes 17:00 - 18:00           2
28   ARQBLO10     Martes 08:00 - 09:00           3
43    ARQBLO2     Jueves 11:00 - 12:00           2
45    ARQBLO2     Jueves 15:00 - 16:00           4
50    ARQBLO2      Lunes 16:00 - 17:00           2
52    ARQBLO2      Lunes 17:00 - 18:00           2
58    ARQBLO2  Miércoles 09:00 - 10:00           2
62    ARQBLO3     Jueves 08:00 - 09:00           3
70    ARQBLO3     Martes 13:00 - 14:00           3
73    ARQBLO3  Miércoles 09:00 - 10:00           2
81    ARQBLO4     Jueves 08:00 - 09:00           2
85    ARQBLO4     Jueves 16:00 - 17:00           2
91    ARQBLO4     Martes 14:00 - 15:00           2
92    ARQBLO4     Martes 17:00 - 18:00           2
95    ARQBLO4  M